In [95]:
import pandas as pd

## On va ajouter de l'Open Data pour avoir les latitudes, longitudes et les régions complètes.

In [96]:
jobs = pd.read_csv('Jobs.csv')

In [97]:
cities = pd.read_csv('cities.csv',sep=';')

In [98]:
cities['ville'] = cities['ville'].str.upper()
jobs['ville'] = jobs['ville'].str.upper()

In [99]:
#on enlève les doublons
cities = cities.drop_duplicates(subset='ville')

In [100]:
nombre_de_doublons = cities.duplicated().sum()
print(nombre_de_doublons)

0


In [101]:
cities

,ville,code_postal,latitude,longitude,department_name,region
0,VILLE DU PONT,25650,46.999873,6.498147,doubs,bourgogne-franche-comté
1,VILLERS GRELOT,25640,47.361512,6.235167,doubs,bourgogne-franche-comté
2,VILLARS LES BLAMONT,25310,47.368384,6.871415,doubs,bourgogne-franche-comté
3,LES VILLEDIEU,25240,46.713906,6.265831,doubs,bourgogne-franche-comté
4,VILLERS BUZON,25170,47.228558,5.852187,doubs,bourgogne-franche-comté
...,...,...,...,...,...,...
39140,THIO,98829,NaN,NaN,nouvelle-calédonie,nouvelle-calédonie
39141,VOH,98833,NaN,NaN,nouvelle-calédonie,nouvelle-calédonie
39142,YATE,98834,NaN,NaN,nouvelle-calédonie,nouvelle-calédonie
39143,SIGAVE,98620,-14.270411,-178.155263,wallis-et-futuna,wallis-et-futuna


In [102]:
#on enlève les doublons
jobs = jobs.drop_duplicates()

In [103]:
nombre_de_doublons = jobs.duplicated().sum()
print(nombre_de_doublons)

0


In [104]:
columns_to_drop = ['commune', 'latitude', 'longitude', 'codePostal']
jobs = jobs.drop(columns=columns_to_drop)

In [105]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 0 to 427
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   intitule                411 non-null    object
 1   description_offre       411 non-null    object
 2   dateCreation            411 non-null    object
 3   typeContrat             411 non-null    object
 4   experienceExige         267 non-null    object
 5   secteurActiviteLibelle  267 non-null    object
 6   qualificationLibelle    237 non-null    object
 7   departement             411 non-null    object
 8   ville                   350 non-null    object
 9   experienceOuiNon        411 non-null    object
 10  entreprise_nom          275 non-null    object
 11  entreprise_description  261 non-null    object
 12  salaire                 212 non-null    object
 13  profils                 144 non-null    object
dtypes: object(14)
memory usage: 48.2+ KB


In [106]:
df = pd.merge(jobs, cities, how='left', on='ville')

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 0 to 410
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   intitule                411 non-null    object 
 1   description_offre       411 non-null    object 
 2   dateCreation            411 non-null    object 
 3   typeContrat             411 non-null    object 
 4   experienceExige         267 non-null    object 
 5   secteurActiviteLibelle  267 non-null    object 
 6   qualificationLibelle    237 non-null    object 
 7   departement             411 non-null    object 
 8   ville                   350 non-null    object 
 9   experienceOuiNon        411 non-null    object 
 10  entreprise_nom          275 non-null    object 
 11  entreprise_description  261 non-null    object 
 12  salaire                 212 non-null    object 
 13  profils                 144 non-null    object 
 14  code_postal             294 non-null    fl

In [108]:
# fusion des colonnes profil et description
# df['nouvelle_colonne'] = df['profils'] + ' ' + df['description_offre']
# df = df.drop(columns=['profils'])

In [109]:
# df.info()

## Transformation variable "Salaire"

In [110]:
import re
import numpy as np

In [111]:
#transformation des donnes type "Annuel de 40,00 Euros à 60,00 Euros" en "Annuel de 40000 Euros à 60000 Euros":
#transformation des données de type 'Mensuel de 1136,00 Euros sur 12 mois':
df['salaire'] = df['salaire'].replace('Mensuel de 1136,00 Euros sur 12 mois', 'Annuel de 13632,00')
df['salaire'] = df['salaire'].replace('Mensuel de 4000,00 Euros à 5000,00 Euros sur 12 mois', 'Annuel de 48000,00 Euros à 60000,00 Euros sur 12 mois')
df['salaire'] = df['salaire'].replace('Annuel de 40,00 Euros à 60,00 Euros', 'Annuel de 40000,00 Euros à 60000,00 Euros')
df['salaire'] = df['salaire'].replace('Annuel de 50,00 Euros à 70,00 Euros', 'Annuel de 50000,00 Euros à 70000,00 Euros')
df['salaire'] = df['salaire'].replace('Annuel de 26000 Euros à 36000 Euros sur 12 mois', 'Annuel de 26000,00 Euros à 36000,00 Euros sur 12 mois')

In [112]:
df['salaire'] = df['salaire'].astype(str)

In [113]:
def extract_salaire_min(salaire):
    match = re.search(r'(\d+,\d+)', salaire)
    return float(match.group(1).replace(',', '.')) if match else None

def extract_salaire_max(salaire):
    match = re.search(r'(\d+,\d+)(?: Euros à (\d+,\d+))?', salaire)
    return float(match.group(2).replace(',', '.')) if match and match.group(2) else None

df['salaire_min_annuel'] = df['salaire'].apply(extract_salaire_min)
df['salaire_max_annuel'] = df['salaire'].apply(extract_salaire_max)


In [114]:
df['salaire_min_annuel'] = pd.to_numeric(df['salaire_min_annuel'].replace('nan', np.nan), errors='coerce')
df['salaire_max_annuel'] = pd.to_numeric(df['salaire_max_annuel'].replace('nan', np.nan), errors='coerce')
df['salaire'] = df['salaire'].replace('nan', np.nan)

In [115]:
#Création de la variable 'salaire_annuel_mean':
df['salaire_annuel_mean'] = np.where(df['salaire_max_annuel'].notnull(),
                                     (df['salaire_min_annuel'] + df['salaire_max_annuel']) / 2,
                                     df['salaire_min_annuel'])

## Renommage des colonnes

In [116]:
# renommer les colonnes (pour la base de donnée)
df = df.rename(columns={'dateCreation':'date_creation', 'qualificationLibelle':'qualification_libelle', 'experienceOuiNon':"experience", 'typeContrat':'type_contrat', 'secteurActiviteLibelle':'secteur_activite', 'codePostal':'code_postal', 'department_name':'departement_nom', 'profils':'profil' })

## Creation de la colonne 'poste'

In [117]:
def categorize_poste(row):
    intitule_lower = row['intitule'].lower()
    if 'data scientist' in intitule_lower or 'ingénieur data scientist' in intitule_lower:
        return 'data scientist'
    elif 'data engineer' in intitule_lower or 'engineer' in intitule_lower or 'ingénieur' in intitule_lower or 'engineering' in intitule_lower :
        return 'data engineer'


# Apply the categorize_poste function to create the 'poste' column
df['poste'] = df.apply(categorize_poste, axis=1)

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 0 to 410
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   intitule                411 non-null    object 
 1   description_offre       411 non-null    object 
 2   date_creation           411 non-null    object 
 3   type_contrat            411 non-null    object 
 4   experienceExige         267 non-null    object 
 5   secteur_activite        267 non-null    object 
 6   qualification_libelle   237 non-null    object 
 7   departement             411 non-null    object 
 8   ville                   350 non-null    object 
 9   experience              411 non-null    object 
 10  entreprise_nom          275 non-null    object 
 11  entreprise_description  261 non-null    object 
 12  salaire                 212 non-null    object 
 13  profil                  144 non-null    object 
 14  code_postal             294 non-null    fl

In [119]:
filtered_df = df.loc[df['poste'].isna(), ['poste', 'intitule', 'description_offre']]
print(filtered_df)
print(filtered_df.shape)

    poste                                           intitule  \
146  None                      Consultant Data - AIX ((H/F))   
147  None  Data Steward - Responsable gestion de données ...   
149  None  STAGE_Marketing strategy and tech analyst F/H ...   
154  None  Stagiaire Project Manager Dashboard Ops e-comm...   
170  None               Data & AI Strategy Manager F/H (H/F)   
172  None                         Développeur Java F/H (H/F)   
178  None                            DATASCIENTIST F/H (H/F)   
190  None  Business data analyste informatique (H/F) (IT)...   
191  None  Data analyste informatique confirmé (IT) / Fre...   
192  None  Expert big data / Hadoop / Cloudera (Rodez) H/...   
193  None  Data analyste informatique (H/F) (IT) / Freelance   
194  None  Data analyste informatiquee DATAIKU (IT) / Fre...   
195  None  Data analyste informatique (H/F) (IT) / Freelance   
199  None                           Ingenieur Big Data (H/F)   
208  None  SAP Master Data Expert F/H - 

In [120]:
# suppression des postes qui ne sont ni data scientist ni data ingénieur
df.dropna(subset=['poste'], inplace=True)

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 376 entries, 0 to 409
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   intitule                376 non-null    object 
 1   description_offre       376 non-null    object 
 2   date_creation           376 non-null    object 
 3   type_contrat            376 non-null    object 
 4   experienceExige         233 non-null    object 
 5   secteur_activite        258 non-null    object 
 6   qualification_libelle   226 non-null    object 
 7   departement             376 non-null    object 
 8   ville                   324 non-null    object 
 9   experience              376 non-null    object 
 10  entreprise_nom          270 non-null    object 
 11  entreprise_description  250 non-null    object 
 12  salaire                 209 non-null    object 
 13  profil                  143 non-null    object 
 14  code_postal             271 non-null    fl

In [122]:
df['poste'].value_counts()

data engineer     238
data scientist    138
Name: poste, dtype: int64

In [123]:
df.to_csv('Jobs_V4.csv', index=False)